In [3]:
import networkit as nk
import affinity_order
import config
from anytree import RenderTree, PreOrderIter
import random

In [4]:
amount_orders = 3
g = affinity_order.load_graph(config.GRAPH_DIR + "uk.graph") #cti

In [5]:
def inverse_ordering(ordering):
    '''
    Inverses the ordering from ordering[pos] == node to ordering[node] == pos.
    '''
    ret = [0] * len(ordering)
    for i in range(len(ordering)):
        ret[ordering[i]] = i
        
    return ret

In [9]:
def export_initial_assim(g, ratio_assim, amount_orders):
    client = nk.gephi.streaming.GephiStreamingClient(url='http://localhost:8080/workspace1')
    #client.exportGraph(g)
    for rep, alg in affinity_order.ORD_ALG.items():
        print(rep)
        orderings = alg(g, amount_orders)
        for i in range(len(orderings)):
            print("i", i)
            client.exportNodeValues(g, color_by_ordering(g, orderings[i], ratio_assim), rep + "_coloring" + str(i))

In [9]:
def export_clusters(g, amount_orders):
    client = nk.gephi.streaming.GephiStreamingClient(url='http://localhost:8080/workspace1')
    root = affinity_order.affinity_tree(g)
    
    level = [x for node in root.children for x in node.children]
    
    nodes = dict()
    for cluster in level:
        nodes[cluster] = set([node_id for node in PreOrderIter(cluster) if node.is_leaf for node_id in node.old_ids])
    
    cluster_color = [0] * nk.graphtools.size(g)[0]
    for i in range(len(level)):
        for node in nodes[level[i]]:
            cluster_color[node] = i
    
    client.exportNodeValues(g, cluster_color, "aff_l2_clusters")
    
    

In [7]:
def export_dfs_ordering(g, coefficients): #changed to export the mappped algebraic distances
    client = nk.gephi.streaming.GephiStreamingClient(url='http://localhost:8080/workspace1')
    #root = affinity_order.affinity_tree(g)
    #random.seed(31415)
    dist = nk.distance.AlgebraicDistance(g)
    dist.preprocess()
    
    client.exportNodeValues(g, inverse_ordering(sort_nodes(map_to_line(dist.getLoads(), coefficients))), "alg-dist")

In [13]:
coefficients = [1]*10
export_dfs_ordering(g, coefficients)

In [11]:
def export_orderings(g, amount_orderings):
    client = nk.gephi.streaming.GephiStreamingClient(url="http://localhost:8080/workspace1")
    # client.exportGraph(g)
    for rep, alg in affinity_order.ORD_ALG.items():
        orderings = alg(g, amount_orders)
        for i in range(len(orderings)):
            client.exportNodeValues(g, inverse_ordering(orderings[i]), rep + "_ordering" + str(i))

In [11]:
def color_by_ordering(g, ordering, ratio_of_side):
    ret = [0] * g.numberOfNodes()
    for i in range(int(ratio_of_side * g.numberOfNodes())):
        ret[ordering[i]] = 1
        ret[ordering[-(i+1)]] = 2
    
    return ret

In [18]:
export_orderings(g, amount_orders)

In [1]:
def sort_nodes(weights):
    '''
    Sorts nodes according to their supplied weights.
    Input: List of weights. weights[n] == w means node n has weight w.
    Output: List of sorted nodes. ret[p] == n means node n has position p in the ordering.
    '''
    
    return [n for w, n in sorted(zip(weights, range(len(weights))))]

In [2]:
def map_to_line(points, coefficients):
    '''
    Maps points to a line. The amount of dimensions is implicitly given by the length of coefficients.
    Points is a list where [i:i+len(coefficients)] are the coordinates of the point i.
    '''
    
    res = list()
    for i in range(0, len(points), len(coefficients)):
        res.append(sum(p*c for p, c in zip(points[i:i+len(coefficients)], coefficients)))
    return res

In [9]:
[1] + [0]*9

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]